Using data from [UCSUSA's satellite database](https://www.ucsusa.org/resources/satellite-database), specifically [UCS-Satellite-Database-8-1-2020.txt](https://www.ucsusa.org/sites/default/files/2020-10/UCS-Satellite-Database-8-1-2020.txt), to allow filtering of satellites for a subsequent hit detection stage.

In [ ]:
import pandas as pd
df = pd.read_csv("metadata/UCS-Satellite-Database-8-1-2020.txt", sep='\t', encoding='L1', low_memory=False) 
df = df.dropna(axis='columns',how='all')
df

In [ ]:
df.info()

The full list of satellites is unwieldy for selecting a single satellite, so first we'll group them into a few hundred shared prefixes, which for a constellation of objects like "Amos 3, Amos 4, Amos 17" would be the shared name "Amos".

In [ ]:
satellites = list(df.iloc[:, 1].unique())
satellites

import re
constellations = sorted(list(set([re.sub(r'[ -].*', '', str(s)) for s in satellites])))
print(len(constellations), ", ".join(constellations))

In [ ]:
import panel as pn, panel.widgets as pnw
pn.extension()

constellation = pnw.Select(options=constellations, name="Constellation")
constellation

In [ ]:
satellite = pnw.Select(options=[s for s in satellites if re.match(constellation.value, str(s))], name="Satellite")
satellite

# Sample of plotting some data from this file

In [ ]:
float_fields = ["Launch Mass (kg.)", "Dry Mass (kg.)"] # etc.
for f in float_fields:
    df[f] = df[f].str.replace('[^0-9]','').astype("float") # Needs more cleanup, e.g. for ranges 1000-2000

In [ ]:
import holoviews as hv, holoviews.operation.datashader as hd
hv.extension("bokeh")
mass_cols=["Launch Mass (kg.)", "Dry Mass (kg.)"]
pts = hv.Points(df[mass_cols], [hv.Dimension("Launch Mass (kg.)", range=(0,5000)), 
                                hv.Dimension("Dry Mass (kg.)",    range=(0,4000))]).opts(width=600)
pts

In [ ]:
pn.Row(constellation, hd.dynspread(hd.rasterize(pts)).opts(width=600)).servable()